In [13]:
import pandas as pd
import pickle
from collections import Counter

In [8]:
# to communicate with google spreadsheet...
import gspread
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk

s = sddk.cloudSession("sciencedata.dk")
# establish connection with gogglesheets...
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict") # or load it from a local storage: json.load(open("../../ServiceAccountsKey.json", "r"))
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
mops_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1VbCIAJssHKV9hlRTwzVFfm40CGnHesq53KXjv2qy4OM/edit?usp=sharing")

endpoint variable has been configured to: https://sciencedata.dk/files/


In [10]:
jstor_df = pd.read_feather("../data/large_files/jstor_df_v1.feather")

In [9]:
trigramCount_cleaned_dict = pickle.load(open("../data/large_files/trigramCount_cleaned_dict.pickle", "rb"))

In [23]:
id = list(trigramCount_cleaned_dict.keys())[0]

In [24]:
trigramCount_cleaned_dict[id]

Counter({'F H MATHEWS': 1,
         'Jesus': 16,
         'lifes through vicissitudes': 1,
         'preparations the their': 1,
         'age': 6,
         'death in man': 1,
         'ambition or uncleanness': 1,
         'lord then your': 1,
         'BD CRIEFF HEPBURN': 1,
         'knotty of points': 1,
         'easy': 3,
         'His acts of': 2,
         'story': 8,
         'Him composed was': 1,
         'do thing': 1,
         'Dr Hunter points': 1,
         'Somme recently the': 1,
         'This consequences obedience': 1,
         'down human into': 1,
         'and fields in': 1,
         'allow take': 1,
         'Sir and speaker': 1,
         'hear': 8,
         'BA BROWN ROBERT': 1,
         'Kingdom supernatural will': 1,
         'at his sentry': 1,
         'shared this truth': 1,
         'God': 40,
         'and twenty went': 1,
         'away nor take': 1,
         'How frightening world': 1,
         'world': 10,
         'And clean me': 1,
         'Jesus pos

In [16]:
def ids_from_colvals(df_name, col, matchstring):
    ids = eval('{0}[{0}["{1}"]{2}]'.format(df_name, col, matchstring))["id"].tolist()
    return ids

def merge_data_from_ids(ids, datadict):
    c = Counter()
    for id in ids:
        d = datadict[id]
        c.update(d)
    return c

def get_tops(df_name, col, matchstring, n=10):
    ids = ids_from_colvals(df_name, col, matchstring)
    c = merge_data_from_ids(ids, unigramCount_dict)
    c_tups = [el for el in c.items() if el[0] != ""]
    top10 = sorted(c_tups, key=lambda kv: kv[1], reverse=True)[:n]
    return top10

decades = []
for decade_n in range(0,10):
    decades.append("19{0}0,19{0}9".format(str(decade_n)))
decades += ["2000,2009", "2010,2019"]
decades

['1900,1909',
 '1910,1919',
 '1920,1929',
 '1930,1939',
 '1940,1949',
 '1950,1959',
 '1960,1969',
 '1970,1979',
 '1980,1989',
 '1990,1999',
 '2000,2009',
 '2010,2019']

In [26]:
# test with function...
ids = ids_from_colvals("jstor_df", "publicationYear", ".between(1920, 1929)")
len(ids)

132

In [27]:
c = merge_data_from_ids(ids, trigramCount_cleaned_dict)

In [29]:
sorted(c.items(), key=lambda kv: kv[1], reverse=True)[:10]

[('have', 3081),
 ('do', 1380),
 ('be', 1229),
 ('time', 1207),
 ('work', 1206),
 ('make', 1180),
 ('say', 1105),
 ('find', 1025),
 ('God', 999),
 ('give', 966)]

In [30]:
def get_tops(df_name, col, matchstring, n=10):
    ids = ids_from_colvals(df_name, col, matchstring)
    c = merge_data_from_ids(ids, trigramCount_cleaned_dict)
    c_tups = [el for el in c.items() if len(el[0].split()) > 1]
    tops = sorted(c_tups, key=lambda kv: kv[1], reverse=True)[:n]
    return tops

In [31]:
for decade in decades:
    print(get_tops("jstor_df", "publicationYear", ".between({0})".format(decade), n=30))

[('New Testament', 1097), ('Old Testament', 783), ('American Journal', 764), ('Christ Jesus', 515), ('Journal Theology', 363), ('hand other', 348), ('same time', 260), ('Epic Indian', 224), ('Biblical World', 217), ('Paul St', 202), ('Paul Saint', 176), ('Fourth Gospel', 172), ('century second', 168), ('School Sunday', 159), ('point view', 157), ('Holy Spirit', 151), ('Man Son', 142), ('States United', 141), ('place take', 134), ('birth virgin', 130), ('New York', 124), ('Jesus life', 120), ('X X', 120), ('Paul apostle', 114), ('J P Peters', 114), ('have seem', 111), ('century first', 107), ('Christ resurrection', 104), ('Apostle Paul', 103), ('Biblical Literature', 102)]
[('New Testament', 821), ('American Journal', 534), ('Old Testament', 313), ('Christ Jesus', 308), ('Paul St', 232), ('Journal Theology', 228), ('same time', 224), ('hand other', 223), ('Biblical World', 216), ('point view', 165), ('place take', 162), ('Apostle Paul', 160), ('Holy Spirit', 156), ('Fourth Gospel', 151)

In [32]:
to_replace = {
    "christian" : "Christian",
    "Biblical" : "biblical",
    "Church" : "church",
    "Faith" : "faith",
    "Apostle" : "apostle",
    "american" : "American",
    "jewish" : "Jewish",
    "old Testament" : "Old Testament",
    "Paul St" : "Paul saint"
}
